In [11]:
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.tools import Tool, StructuredTool
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [7]:
model = ChatGroq(
    model="llama3-8b-8192"
)

In [8]:
model.invoke("What model are you")

AIMessage(content="I am LLaMA, a large language model trained by Meta AI that can understand and respond to human input in a conversational manner. I'm a type of AI designed to simulate conversation, answer questions, and even generate text. I'm constantly learning and improving, so please bear with me if I make any mistakes. How can I help you today?", response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 14, 'total_tokens': 87, 'completion_time': 0.057870884, 'prompt_time': 0.002837743, 'queue_time': None, 'total_time': 0.060708627}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-460191ec-129a-436c-97ea-91ab6f60c8cb-0')

In [9]:
def greet_user(name: str) -> str:
    """Greets the user by name."""
    return f"Hello {name}"

def reverse_string(text: str) -> str:
    """Reverses the input string."""
    return text[::-1]

def concatenate_strings(text1: str, text2: str) -> str:
    """Concatenates two strings."""
    return text1 + text2

In [10]:
# Pydantic model for tool arguments

class ConcatenateStringsArgs(BaseModel):
    text1: str = Field(description="First string")
    text2: str = Field(description="Second string")

In [25]:
tools = [
    Tool(
        name="greet_user",
        func=greet_user,
        description="use this tool only when to greet someone",
    ),
    Tool(
        name="reverse_string",
        func=reverse_string,
        description="only useful when user wants to reverse the input string",
    ),
    StructuredTool.from_function(
        name="concatenate_strings",
        func=concatenate_strings,
        description="Concatenates two strings",
        input_model=ConcatenateStringsArgs,
    )
]

In [14]:
prompt = hub.pull("hwchase17/openai-tools-agent")

In [26]:
agent = create_tool_calling_agent(
    tools=tools,
    llm=model,
    prompt=prompt
)

In [27]:
agent_executor = AgentExecutor(
    agent=agent,
    verbose=True,
    tools=tools,
    handle_parsing_errors=True
)

In [28]:
agent_executor.invoke({
    "input": "Greet Manoj"
})



> Entering new AgentExecutor chain...

Invoking: `greet_user` with `Manoj`


Hello ManojHello Manoj!

> Finished chain.


{'input': 'Greet Manoj', 'output': 'Hello Manoj!'}

In [30]:
agent_executor.invoke({
    "input": "reverse Manoj"
})



> Entering new AgentExecutor chain...

Invoking: `reverse_string` with `Manoj`


jonaMjonaM

> Finished chain.


{'input': 'reverse Manoj', 'output': 'jonaM'}

In [29]:
agent_executor.invoke({
    "input": "Concatenate these strings: hello, world"
})



> Entering new AgentExecutor chain...

Invoking: `concatenate_strings` with `{'text1': 'hello', 'text2': ' world'}`


hello worldHello world!

> Finished chain.


{'input': 'Concatenate these strings: hello, world', 'output': 'Hello world!'}